# Projeto de Modelagem: Qualidade de Vinhos Tintos (Wine Quality - Red)

**Disciplina:** Modelagem de Dados — 2º Bimestre  
**Dataset:** Wine Quality (UCI) — vinhos tintos  
**Objetivo:** construir um pipeline completo de Data Science com EDA, limpeza, modelos de regressão e classificação, avaliação e otimização.

---

## Hipótese / Pergunta de negócio
Quais características físico‑químicas mais influenciam a nota de qualidade (`quality`) dos vinhos tintos?  
Além disso, é possível classificar vinhos em **bons** (≥7) e **não bons** (≤6) com boa performance?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, classification_report, confusion_matrix,
    roc_auc_score
)

warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid")
np.random.seed(42)

## 1. Funções auxiliares
- **IQR** para limites de outliers  
- **Matriz de confusão** para classificação

In [ ]:
def outlier_limits_iqr(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5 * iqr
    high = q3 + 1.5 * iqr
    return low, high

def plotar_matriz_confusao(y_real, y_pred, titulo):
    plt.figure(figsize=(6,4))
    sns.heatmap(confusion_matrix(y_real, y_pred),
                annot=True, fmt="d", cmap="Blues")
    plt.title(titulo)
    plt.xlabel("Previsto")
    plt.ylabel("Real")
    plt.show()

## 2. Carregamento e limpeza

**Etapas realizadas:**
1. Leitura do CSV (separador `;`).  
2. Padronização de nomes de colunas (substitui espaço por `_`).  
3. Remoção de duplicatas.  
4. Tratamento de outliers por *winsorization* (clip IQR) **somente nas variáveis explicativas**, preservando a variável‑alvo.

In [ ]:
df = pd.read_csv("winequality-red.csv", sep=";")

# padronizar nomes
df.columns = df.columns.str.replace(" ", "_")

# remover duplicatas
shape_antes = df.shape
df = df.drop_duplicates()
print(f"Duplicatas removidas: {shape_antes} -> {df.shape}")

# tratar outliers (winsorization) menos no target
TARGET = "quality"
cols = [c for c in df.columns if c != TARGET]

for col in cols:
    low, high = outlier_limits_iqr(df[col])
    df[col] = df[col].clip(low, high)

df.head()

## 3. Análise Exploratória (EDA)

Nesta etapa verificamos:
- estrutura do dataset (tipos, faixas, estatísticas)  
- correlações gerais e com o alvo  
- relações bivariadas entre variáveis mais relevantes e a qualidade

In [ ]:
df.info()
df.describe()

In [ ]:
plt.figure(figsize=(10,6))
corr = df.corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Mapa de Correlação")
plt.show()

corr["quality"].sort_values(ascending=False)

In [ ]:
cols_pair = ["alcohol", "sulphates", "volatile_acidity", "quality"]
sns.pairplot(df[cols_pair], hue="quality", corner=True)
plt.show()

# PARTE 1 — REGRESSÃO

Vamos prever `quality` como variável numérica.

Modelos exigidos:
1. **Regressão linear simples**  
2. **Regressão linear múltipla**  
3. **Regressão polinomial**

## 4. Preparação de dados (split + scaling)

- Separação de features (`X`) e alvo (`y`).  
- Divisão treino/teste (70/30).  
- Padronização com `StandardScaler`.

In [ ]:
X = df.drop(columns=["quality"])
y = df["quality"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

X_train_sc.shape, X_test_sc.shape

## 5.1 Regressão linear simples (apenas `alcohol`)

Usamos `statsmodels` para interpretar coeficientes e significância estatística.

In [ ]:
X_simple = df[["alcohol"]]
y_simple = df["quality"]

X_tr_s, X_te_s, y_tr_s, y_te_s = train_test_split(
    X_simple, y_simple, test_size=0.30, random_state=42
)

X_tr_s_sm = sm.add_constant(X_tr_s)
model_simple = sm.OLS(y_tr_s, X_tr_s_sm).fit()
print(model_simple.summary())

## 5.2 Regressão linear múltipla

Treinamos com todas as variáveis e avaliamos com **MAE, RMSE e R²**.

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train_sc, y_train)

y_pred_lin = lin_reg.predict(X_test_sc)

mae_lin  = mean_absolute_error(y_test, y_pred_lin)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))
r2_lin   = r2_score(y_test, y_pred_lin)

print(f"Linear múltipla -> MAE: {mae_lin:.4f} | RMSE: {rmse_lin:.4f} | R²: {r2_lin:.4f}")

## 5.3 Regressão polinomial (grau 2)

Expandimos features para capturar não‑linearidades.

In [ ]:
poly_model = Pipeline([
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("lr", LinearRegression())
])

poly_model.fit(X_train_sc, y_train)
y_pred_poly = poly_model.predict(X_test_sc)

mae_poly  = mean_absolute_error(y_test, y_pred_poly)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))
r2_poly   = r2_score(y_test, y_pred_poly)

print(f"Polinomial grau 2 -> MAE: {mae_poly:.4f} | RMSE: {rmse_poly:.4f} | R²: {r2_poly:.4f}")

## 6. Diagnóstico de resíduos (modelo linear)

Verificamos se os resíduos parecem centrados em 0 e aproximadamente normais.

In [ ]:
residuos = y_test - y_pred_lin

plt.figure(figsize=(6,4))
plt.scatter(y_pred_lin, residuos, alpha=0.6)
plt.axhline(0, linestyle="--")
plt.title("Resíduos vs Valores Preditos")
plt.xlabel("Predito")
plt.ylabel("Resíduo")
plt.show()

plt.figure(figsize=(6,4))
sns.histplot(residuos, kde=True)
plt.title("Distribuição dos resíduos")
plt.show()

## 7. Validação cruzada (regressão)

Avaliamos estabilidade do modelo linear com CV (5 folds).

In [ ]:
cv_r2 = cross_val_score(lin_reg, X_train_sc, y_train, cv=5, scoring="r2")
print("R² CV:", cv_r2)
print("R² médio:", cv_r2.mean())

# PARTE 2 — CLASSIFICAÇÃO

Criamos uma variável binária:
- **good = 1:** `quality ≥ 7`  
- **good = 0:** `quality ≤ 6`

Modelos exigidos:
1. **Naive Bayes Gaussiano**  
2. **Regressão Logística**

## 8. Preparação para classificação

- Criar alvo binário `good`.  
- Split treino/teste estratificado (desbalanceamento).  
- Padronização.

In [ ]:
df["good"] = (df["quality"] >= 7).astype(int)

Xc = df.drop(columns=["quality", "good"])
yc = df["good"]

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    Xc, yc, test_size=0.30, random_state=42, stratify=yc
)

scaler_c = StandardScaler()
X_train_c_sc = scaler_c.fit_transform(X_train_c)
X_test_c_sc  = scaler_c.transform(X_test_c)

yc.value_counts(normalize=True)

## 9.1 Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train_c_sc, y_train_c)

y_pred_nb = nb.predict(X_test_c_sc)

print("Naive Bayes accuracy:", accuracy_score(y_test_c, y_pred_nb))
print(classification_report(y_test_c, y_pred_nb, zero_division=0))
plotar_matriz_confusao(y_test_c, y_pred_nb, "Matriz de Confusão - Naive Bayes")

## 9.2 Regressão Logística (baseline)

Avaliamos **accuracy, AUC, precision, recall e F1**.

In [ ]:
log_reg = LogisticRegression(max_iter=3000)
log_reg.fit(X_train_c_sc, y_train_c)

y_pred_log = log_reg.predict(X_test_c_sc)
y_prob_log = log_reg.predict_proba(X_test_c_sc)[:,1]

print("Logística accuracy:", accuracy_score(y_test_c, y_pred_log))
print("AUC:", roc_auc_score(y_test_c, y_prob_log))
print(classification_report(y_test_c, y_pred_log, zero_division=0))
plotar_matriz_confusao(y_test_c, y_pred_log, "Matriz de Confusão - Logística (baseline)")

## 10. Otimização com GridSearchCV (Logística)

Buscamos melhores hiperparâmetros para aumentar F1.

In [ ]:
param_grid = {
    "C": [0.01, 0.1, 1, 10],
    "solver": ["lbfgs", "liblinear"]
}

grid = GridSearchCV(
    LogisticRegression(max_iter=4000),
    param_grid,
    cv=5,
    scoring="f1"
)

grid.fit(X_train_c_sc, y_train_c)

print("Melhores parâmetros:", grid.best_params_)
print("Melhor F1 (CV):", grid.best_score_)

## 11. Avaliação final do modelo otimizado

In [ ]:
best_log = grid.best_estimator_

y_pred_best = best_log.predict(X_test_c_sc)
y_prob_best = best_log.predict_proba(X_test_c_sc)[:,1]

print("=== RESULTADO FINAL ===")
print("Accuracy:", accuracy_score(y_test_c, y_pred_best))
print("AUC:", roc_auc_score(y_test_c, y_prob_best))
print(classification_report(y_test_c, y_pred_best, zero_division=0))

plotar_matriz_confusao(y_test_c, y_pred_best, "Matriz de Confusão - Logística Otimizada")

## 12. Conclusões (preencha com seus resultados)

- Compare o desempenho dos três modelos de regressão (MAE/RMSE/R²).  
- Descreva quais variáveis parecem mais importantes (correlação + regressão simples).  
- Compare NB vs Logística (baseline e otimizada).  
- Comente limitações e possíveis melhorias (ex.: modelos não lineares, balanceamento, mais dados).